# __Hands-on with TensorFlow: Part A__
 This demo, let's see how TensorFlow works and build a deep neural network model using the MNIST dataset.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
print('TensorFlow version:', tf.__version__)


TensorFlow version: 2.15.0


In [3]:
# Load the MNIST dataset and normalize the input data by dividing the train and test sets by 255

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
mnist

<module 'keras.api._v2.keras.datasets.mnist' from 'C:\\Users\\srika\\anaconda3\\Lib\\site-packages\\keras\\api\\_v2\\keras\\datasets\\mnist\\__init__.py'>

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [6]:
# Create a Sequential model with Flatten, Dense, and Dropout layers

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [7]:
model

In [8]:
# Get the predictions from the model using the train data for one column, as the optimizer is not yet applied

predictions = model(x_train[:1]).numpy()
predictions

array([[-0.12614325, -0.648785  , -0.01115082,  0.46964863,  1.0326533 ,
         0.55589056, -0.7938734 , -0.06033453,  0.7030475 , -0.5100605 ]],
      dtype=float32)

In [9]:
print(predictions.shape)

(1, 10)


In [10]:
# Apply softmax activation to the predictions and print the output in terms of probabilities
tf.nn.softmax(predictions).numpy()

array([[0.07019109, 0.04161995, 0.07874492, 0.12735942, 0.2236356 ,
        0.13883069, 0.035999  , 0.07496566, 0.16084033, 0.04781331]],
      dtype=float32)

The predicted values in terms of probabilities are displayed as an array

In [11]:
# Let's create a loss function for the model.
# Next, let's define the SparseCategoricalCrossentropy loss function.

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1], predictions).numpy()

In [14]:
# Now, compile the model with the Adam optimizer, the loss function, and the accuracy metric

model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])

Here, we have used an optimizer as Adam, loss as a loss function, and metrics as accuracy

In [15]:
# Fit the model using the training data for 5 epochs

model.fit(x_train, y_train, epochs=5)

Epoch 1/5


1875/1875 [==============================] - 12s 5ms/step - loss: 0.2982 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1434 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1099 - accuracy: 0.9667
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0885 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0755 - accuracy: 0.9764


In [16]:
# The model fits the data.
# Here, we can see that the accuracy is increased by different epochs.

In [17]:
# Evaluate the model using the testing data and print the results

model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0775 - accuracy: 0.9763 - 1s/epoch - 4ms/step


[0.07754295319318771, 0.9763000011444092]

The accuracy score is 0.977, and the loss is 0.072.

In [18]:
# Create a probability model by adding the Softmax layer to the existing model

probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])

In [19]:
# And, get the predictions using the test data

probability_model(x_test[:3])

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[1.2002215e-07, 2.5601892e-09, 1.4977709e-05, 7.1184878e-04,
        5.8060054e-12, 3.7825866e-06, 7.6441098e-14, 9.9922514e-01,
        2.4704195e-07, 4.3896503e-05],
       [2.5490248e-09, 2.6897891e-05, 9.9997008e-01, 2.9986468e-06,
        1.2044564e-15, 3.8867491e-08, 4.8659685e-09, 1.6041117e-13,
        3.9134216e-09, 4.1151862e-17],
       [6.5699203e-07, 9.9873978e-01, 4.7523883e-04, 3.1737563e-05,
        5.7500311e-06, 2.1959537e-05, 2.1187077e-06, 3.4510679e-04,
        3.7758099e-04, 6.4688606e-08]], dtype=float32)>

Here, we have the probabilities for the first three observations and different classes.

# __Hands-on with TensorFlow: Part B__

- In this demo, let's see how to implement ensemble techniques using Keras and sklearn.

In [20]:
# Import necessary libraries from Sklearn, TensorFlow, and Keras
# Load the dataset using sklearn's make_moons function

import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [21]:
# Define X and y and split the dataset into training and testing sets using the train_test_split function

X,y=make_moons(n_samples=500,noise=0.30,random_state=42)

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [22]:
# Define Logistic Regression, Random Forest, Support Vector Machine, and Keras Neural Network classifiers

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42,probability=True)

In [23]:
# Import Sequential and Dense modules from the TensorFlow library to create a neural network

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Define the function build_nn() to build a neural network

def build_nn():
    model = Sequential([
        Dense(50, activation='relu', input_shape=[2]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [25]:
# Now, let's wrap scikit learn and build a neural network with 500 epochs with the Keras Classifier.

import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

keras_clf = KerasClassifier(build_fn=build_nn, epochs=500, verbose=False)

Scikit learn is wrapped.

In [26]:
keras_clf._estimator_type = "classifier"

In [27]:
# Combine the classifiers using sklearn's VotingClassifier with soft voting

voting = VotingClassifier(estimators=[('lr', log_clf),('rf', rnd_clf),
                                      ('svc', svm_clf),('keras',keras_clf)],
                          voting='soft',flatten_transform=True)

- The classifiers are combined with soft voting.

In [28]:
# Train the voting classifier and each individual classifier

voting = VotingClassifier(estimators=[('Keras', KerasClassifier(model=build_nn, epochs=500, verbose=False))])
voting.fit(X_train, y_train)

VotingClassifier(estimators=[('Keras',
                              KerasClassifier(epochs=500, model=<function build_nn at 0x0000028803DFBA60>, verbose=False))])

In [29]:
# Consider voting with VotingClassifier and estimator
# The first is LogisticRegression, the second is RandomForest, the third is SVM, the fourth is KerasClassifier, and the fifth is VotingClassifier.

for clf in (log_clf, rnd_clf, svm_clf, keras_clf, voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) # Make predictions on the test set and calculate the accuracy for each classifier
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896


C:\Users\srika\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


KerasClassifier 0.888
VotingClassifier 0.896


As shown above, LogisticRegression is at 86% accuracy, RandomForest is at 89%, SVM is at 89%, KerasClassifier is at 89%, and VotingClassifier is at 89%.

###### Basics of Keras and TensorFlow concluded